<a href="https://colab.research.google.com/github/Buuuuli/AIPI540_recommendation/blob/main/DL_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from os.path import join
from google.colab import drive

ROOT = "/content/drive"
drive.mount(ROOT, force_remount=False)

Mounted at /content/drive


In [2]:
%cd /content/drive/"My Drive"/AIPI540_recom/data

/content/drive/My Drive/AIPI540_recom/data


In [3]:
import sys
import os
import torch
import cornac
import papermill as pm
import scrapbook as sb
import pandas as pd
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED

print("System version: {}".format(sys.version))
print("PyTorch version: {}".format(torch.__version__))
print("Cornac version: {}".format(cornac.__version__))

ModuleNotFoundError: ignored

In [4]:
!pip install recommenders
!pip install cornac
!pip install papermill
!pip install scrapbook

     |████████████████████████████████| 335 kB 14.6 MB/s 
     |████████████████████████████████| 310 kB 82.7 MB/s 
     |████████████████████████████████| 197 kB 68.3 MB/s 
     |████████████████████████████████| 1.5 MB 98.0 MB/s 
     |████████████████████████████████| 636 kB 83.6 MB/s 
     |████████████████████████████████| 11.8 MB 54.7 MB/s 
     |████████████████████████████████| 3.8 MB 66.2 MB/s 
     |████████████████████████████████| 61 kB 9.8 MB/s 
     |████████████████████████████████| 12.4 MB 83.2 MB/s 
     |████████████████████████████████| 749 kB 77.6 MB/s 
     |████████████████████████████████| 10.9 MB 69.6 MB/s 
     |████████████████████████████████| 380 kB 98.2 MB/s 
     |████████████████████████████████| 6.5 MB 76.4 MB/s 
     |████████████████████████████████| 895 kB 67.6 MB/s 
     |████████████████████████████████| 77 kB 7.5 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=706174 sha256=f0a6bfddb9c6abfac3e0fe00d8297b4bf

In [ ]:
df = pd.read_pickle('final_dataframe.pkl')

In [ ]:
df_new = df.sample(20000)

In [ ]:
data_encoded = df_new.copy()
for col in ['business_id','user_id']:
    data_encoded[col] = data_encoded[col].astype('category') # Convert to category type
    data_encoded[col] = data_encoded[col].cat.codes # Convert to numerical code
data_encoded.head()

,business_id,name,address,city,state,postal_code,stars,review_count,categories,user_id,review_stars
539321,3834,Oxford Exchange,420 W Kennedy Blvd,Tampa,FL,33606,4.5,1868,"Restaurants, American (New), Coffee & Tea, Boo...",5133,5
497165,3005,Bern's Steak House,1208 S Howard Ave,Tampa,FL,33606,4.5,2924,"Arts & Entertainment, Steakhouses, Restaurants",12403,2
426008,99,Benjarong Thai,14402 N Dale Mabry Hwy,Tampa,FL,33618,4.0,113,"Restaurants, Vegetarian, Asian Fusion, Thai",4366,5
172243,1944,Rumba Island Bar & Grill,1800 Gulf To Bay Blvd,Clearwater,FL,33765,4.0,771,"Bars, Nightlife, Restaurants, Barbeque, Caribb...",8001,5
310441,1158,Villa Gallace Italian Restaurant,109 Gulf Blvd,Indian Rocks Beach,FL,33785,4.5,344,"Italian, Restaurants",15580,5


In [ ]:
data_encoded_only = data_encoded[['business_id','user_id','review_stars']]

In [ ]:
data_encoded_only = data_encoded_only.reset_index(drop=True)

In [ ]:
data_encoded_only

,business_id,user_id,review_stars
0,3834,5133,5
1,3005,12403,2
2,99,4366,5
3,1944,8001,5
4,1158,15580,5
...,...,...,...
19995,10,9410,4
19996,2148,1096,5
19997,4305,12647,4
19998,2202,1770,5


In [ ]:
data_encoded_only.rename(columns = {'business_id':'itemID', 'user_id':'userID','review_stars':'rating'}, inplace = True)

In [ ]:
train, test = python_random_split(data_encoded_only, 0.75)

In [ ]:
# top k items to recommend
TOP_K = 10

# Model parameters
LATENT_DIM = 5
ENCODER_DIMS = [300]
ACT_FUNC = "tanh"
LIKELIHOOD = "gaus"
NUM_EPOCHS = 500
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [ ]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 4131
Number of items: 12662


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 12 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


In [ ]:
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
with Timer() as t:
    all_predictions = predict_ranking(bivae, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

In [ ]:
all_predictions

In [ ]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')